In [1]:
import pandas as pd

In [2]:
filename = '/content/drive/Othercomputers/My Laptop/sales_predictions/sales_predictions.csv'

In [3]:
#Import a csv file
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [5]:
#Decided to drop Item_Weight column
df.drop(columns = ['Item_Weight'], inplace=True)

In [6]:
#Content conscious customers
#might purchase based on this description, plus field can be converted into 0 
#for regular and 1 for Low Fat. Arriving at two values for this column is a 
#good thing
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [7]:
#Performing the replace on the improper category descriptions
df.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'}},inplace=True)

In [8]:
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [9]:
#Checking the Item Type for any necessary category item cleanup
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [10]:
#The next three cells are ran to see when Grouped by Outlet Location Type, Outlet_Type
#Outlet Establishment Year, which have no Outlet Size, the mean, min and max are
#presented to information the judgement taken on how to fill in the missing
#Outlet Size 
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].mean()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small               340.329723
                     Supermarket Type1 1997                      Small              2277.844267
                                       1999                      Medium             2348.354635
Tier 2               Supermarket Type1 2002                      NaN                2192.384798
                                       2004                      Small              2438.841866
                                       2007                      NaN                2340.675263
Tier 3               Grocery Store     1998                      NaN                 339.351662
                     Supermarket Type1 1987                      High               2298.995256
                     Supermarket Type2 2009                      Medium             1995.498739
                     Supermarket Type3 1985                      Medium             3694.038558

In [11]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].min()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small                  33.9558
                     Supermarket Type1 1997                      Small                 101.8674
                                       1999                      Medium                111.8544
Tier 2               Supermarket Type1 2002                      NaN                    99.8700
                                       2004                      Small                 113.8518
                                       2007                      NaN                   143.8128
Tier 3               Grocery Store     1998                      NaN                    33.2900
                     Supermarket Type1 1987                      High                   73.2380
                     Supermarket Type2 2009                      Medium                 69.2432
                     Supermarket Type3 1985                      Medium                241.6854

In [12]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].max()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small                1482.0708
                     Supermarket Type1 1997                      Small                9779.9362
                                       1999                      Medium               7646.0472
Tier 2               Supermarket Type1 2002                      NaN                  8994.9580
                                       2004                      Small                8479.6288
                                       2007                      NaN                  9664.7528
Tier 3               Grocery Store     1998                      NaN                  1775.6886
                     Supermarket Type1 1987                      High                10256.6490
                     Supermarket Type2 2009                      Medium               6768.5228
                     Supermarket Type3 1985                      Medium              13086.9648

###Based on the presented data, I have made the judgement to update the missing data for Tier 2, Supermarket Type 1, 2002 to Outlet size Small
###Tier 2, Supermarket Type 1, 2007 to Outlet size Medium
###Tier 3, Grocery Store, 1998 to Outlet size Small

In [13]:
outlet_established_year1998_filter = df['Outlet_Establishment_Year'] == 1998

In [14]:
outlet_established_year2002_filter = df['Outlet_Establishment_Year'] == 2002

In [15]:
outlet_established_year2007_filter = df['Outlet_Establishment_Year'] == 2007

In [16]:
#Code updates cells in column Outlet Size with the determined valuedf.loc[outlet_established_year1998_filter, 'Outlet_Size'].fillna("Small")

In [17]:
df.loc[outlet_established_year1998_filter,'Outlet_Size'] = df.loc[outlet_established_year1998_filter, 'Outlet_Size'].fillna("Small")

In [18]:
df.loc[outlet_established_year2002_filter, 'Outlet_Size'] = df.loc[outlet_established_year2002_filter, 'Outlet_Size'].fillna("Small")

In [19]:
df.loc[outlet_established_year2007_filter, 'Outlet_Size'] = df.loc[outlet_established_year2007_filter, 'Outlet_Size'].fillna("Medium")

In [20]:
df.describe()

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8523.000000,8523.000000,8523.000000,8523.000000
mean,0.066132,140.992782,1997.831867,2181.288914
std,0.051598,62.275067,8.371760,1706.499616
min,0.000000,31.290000,1985.000000,33.290000
25%,0.026989,93.826500,1987.000000,834.247400
50%,0.053931,143.012800,1999.000000,1794.331000
75%,0.094585,185.643700,2004.000000,3101.296400
max,0.328391,266.888400,2009.000000,13086.964800


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Fat_Content           8523 non-null   object 
 2   Item_Visibility            8523 non-null   float64
 3   Item_Type                  8523 non-null   object 
 4   Item_MRP                   8523 non-null   float64
 5   Outlet_Identifier          8523 non-null   object 
 6   Outlet_Establishment_Year  8523 non-null   int64  
 7   Outlet_Size                8523 non-null   object 
 8   Outlet_Location_Type       8523 non-null   object 
 9   Outlet_Type                8523 non-null   object 
 10  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(3), int64(1), object(7)
memory usage: 732.6+ KB


#First Checkpoint
##Checkpoint is provided just before performing removal of any duplicate rows

In [22]:
df1_copy = df

In [23]:
df.drop_duplicates(inplace=True)